<a href="https://colab.research.google.com/github/MustafaGhaziTalab/Homwork/blob/main/Mustafa_Ghazi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [3]:
!ls "/content/drive/My Drive/MMM"


b.csv  student_records.csv


In [4]:
import pandas as pd
from sortedcontainers import SortedDict
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the CSV file
data_path = '/content/drive/MyDrive/MMM/student_records.csv'

# Load data from CSV into a DataFrame
def load_data(csv_path):
    data = pd.read_csv(csv_path)
    return data

# Function to display the structure of the data
def analyze_data_structure(data):
    print("Column Names:", data.columns.tolist())
    print("First Few Rows of Data:")
    print(data.head())

# Create a B-tree from the data
def create_btree_from_data(data):
    b_tree = SortedDict()
    for index, row in data.iterrows():
        # Convert 'MATRIC NO' to text before using it as a key
        matric_no = str(row['MATRIC NO'])
        b_tree[matric_no] = row.to_dict()
    return b_tree


# Load the data
data = load_data(data_path)

# Analyze the data structure
analyze_data_structure(data)

# Create the B-tree
student_btree = create_btree_from_data(data)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Column Names: ['MATRIC NO', 'NAME STYLE', 'CODE', 'COURSE TITLE', 'LEARNING METHOD', 'SECTION', 'CREDIT STATUS']
First Few Rows of Data:
   MATRIC NO  NAME STYLE         CODE  \
0  MCS000000  JWJMH RPUN     MECR1063   
1  MCS000000  JWJMH RPUN     MECS1043   
2  MCS000000  JWJMH RPUN     MECS1023   
3  MCS000000  JWJMH RPUN     UECS6023   
4  MCS000000  JWJMH RPUN  UECS6013-IT   

                                 COURSE TITLE LEARNING METHOD SECTION  \
0                   Cryptographic Engineering              FC       1   
1    Research Methodology in Computer Science              FC       1   
2      Advanced Data Structure and Algorithms              FC       1   
3  Introduction to Technical Entrepreneurship              FC       1   
4                          Project Management              FC       1   

  CREDIT STATUS  
0             3  
1           

In [6]:

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the CSV file
data_path = '/content/drive/MyDrive/MMM/student_records.csv'

# Load data from CSV into a DataFrame
def load_data(csv_path):
    return pd.read_csv(csv_path)

# Load the data
data = load_data(data_path)

# Create the widgets for the interface
search_input = widgets.Text(placeholder='Enter MATRIC NO or NAME', description='Search:')
add_inputs = {column: widgets.Text() for column in data.columns}
edit_inputs = {column: widgets.Text() for column in data.columns}
delete_input = widgets.Text(description='MATRIC NO:')
delete_course_matric_input = widgets.Text(description='MATRIC NO:')
delete_course_title_input = widgets.Text(description='Course Title:')
delete_course_button = widgets.Button(description='Delete Course', button_style='danger')
delete_course_output = widgets.Output()

search_button = widgets.Button(description='Search', button_style='success')
add_button = widgets.Button(description='Add', button_style='info')
edit_button = widgets.Button(description='Update', button_style='warning')
delete_button = widgets.Button(description='Delete', button_style='danger')

output_area = widgets.Output()

# Function to refresh the output area with new data
def refresh_output(new_data):
    with output_area:
        clear_output()
        display(new_data)

# Define the button click actions
def on_search_clicked(b):
    with output_area:
        clear_output()
        search_query = search_input.value.lower()
        if search_query:
            results = data[
                data['MATRIC NO'].str.contains(search_query, case=False, na=False) |
                data['NAME STYLE'].str.contains(search_query, case=False, na=False)
            ]
            if not results.empty:
                display(results)
            else:
                print("No records found.")
        else:
            print("Please enter a search query.")

def on_add_clicked(b):
    new_data = {key: val.value for key, val in add_inputs.items()}
    if new_data['MATRIC NO'] not in data['MATRIC NO'].values:
        data.loc[len(data)] = new_data
        refresh_output(data)
        for widget in add_inputs.values():
            widget.value = ''  # Clear the input after adding
    else:
        print(f"Matric No {new_data['MATRIC NO']} already exists.")

def on_edit_clicked(b):
    with output_area:
        clear_output()
        matric_no = edit_inputs['MATRIC NO'].value  # Assuming the matric no is not changed

        # Ensure that the MATRIC NO exists in the DataFrame
        if matric_no in data['MATRIC NO'].values:
            # Get the index of the student in the DataFrame
            student_index = data.index[data['MATRIC NO'] == matric_no].tolist()[0]

            # Update the student's information
            for key, widget in edit_inputs.items():
                if key != 'MATRIC NO':  # Avoid changing the MATRIC NO
                    data.at[student_index, key] = widget.value if widget.value else data.at[student_index, key]

            # Save the updated DataFrame back to CSV
            data.to_csv(data_path, index=False)

            print(f"Student record for MATRIC NO {matric_no} has been updated.")
        else:
            print(f"No student found with MATRIC NO {matric_no}.")

def on_delete_clicked(b):
    matric_no = delete_input.value
    if matric_no in data['MATRIC NO'].values:
        data.drop(data[data['MATRIC NO'] == matric_no].index, inplace=True)
        refresh_output(data)
    else:
        print(f"No student found with MATRIC NO {matric_no}.")

def on_delete_course_button_clicked(b):
    matric_no = delete_course_matric_input.value.strip()
    course_title = delete_course_title_input.value.strip()
    with delete_course_output:
        clear_output()
        if matric_no and course_title:
            delete_student_course(matric_no, course_title)
        else:
            print("Please enter both MATRIC NO and Course Title.")

# Function to delete a specific course for a specific student
def delete_student_course(matric_no, course_title):
    global data  # Use global data
    # Verify the presence of the student and the course
    if matric_no in data['MATRIC NO'].values:
        student_courses = data[data['MATRIC NO'] == matric_no]
        if course_title in student_courses['COURSE TITLE'].values:
            # Delete the course
            data = data[~((data['MATRIC NO'] == matric_no) & (data['COURSE TITLE'] == course_title))]
            #Save the changes to the file
            data.to_csv(data_path, index=False)
            print(f"Course '{course_title}' has been deleted for student with MATRIC NO '{matric_no}'.")
        else:
            print(f"Course title '{course_title}' not found for student with MATRIC NO '{matric_no}'.")
    else:
        print(f"No student found with MATRIC NO '{matric_no}'.")

# Bind the buttons to their respective functions
search_button.on_click(on_search_clicked)
add_button.on_click(on_add_clicked)
edit_button.on_click(on_edit_clicked)
delete_button.on_click(on_delete_clicked)
delete_course_button.on_click(on_delete_course_button_clicked)

# Arrange the interface
search_section = widgets.VBox([search_input, search_button])
add_section = widgets.VBox([widgets.VBox([widgets.Label(key), add_inputs[key]]) for key in data.columns] + [add_button])
edit_section = widgets.VBox([widgets.VBox([widgets.Label(key), edit_inputs[key]]) for key in data.columns] + [edit_button])
delete_section = widgets.VBox([delete_input, delete_button])
course_delete_section = widgets.VBox([
    delete_course_matric_input,
    delete_course_title_input,
    delete_course_button,
    delete_course_output
])

interface = widgets.Tab([search_section, add_section, edit_section, delete_section, course_delete_section])
interface.set_title(0, 'Search')
interface.set_title(1, 'Add')
interface.set_title(2, 'Edit')
interface.set_title(3, 'Delete')
interface.set_title(4, 'Delete Course')


# Display the interface
display(interface, output_area)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Output()